In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
import os

In [3]:
# Load relevant data
associations = pd.read_csv('associations_rich.csv')
hours = pd.read_csv('hours_rich.csv')
hours['day'] = pd.to_datetime(hours['day'])
incidences = pd.read_csv('incidences_rich.csv')

In [4]:
'''
 that all of our sample data is loaded, we can begin our analysis.
 Lets begin with a timeline of the planned and effective
 allocation of all workers.
'''

# Join hours and associations to determine whether or not
# the planned hours occured in the worker's original workplace.

hours_associations = pd.merge(hours, associations, how='left',on='person_id')

In [5]:
hours_associations.dtypes

day             datetime64[ns]
hour                     int64
person_id                int64
worked_hours           float64
from_date               object
to_date                 object
store_id                 int64
dtype: object

In [6]:
hours_associations['day'] = pd.to_datetime(hours_associations['day'])
hours_associations['from_date'] = pd.to_datetime(hours_associations['from_date'])
hours_associations['to_date'] = hours_associations['to_date'].fillna(datetime.datetime.today().date())
hours_associations['to_date'] = pd.to_datetime(hours_associations['to_date'])

hours_associations

,day,hour,person_id,worked_hours,from_date,to_date,store_id
0,2023-11-02,6,665917,1.0,2017-06-02,2021-04-30,1
1,2023-11-02,6,665917,1.0,2023-07-01,2024-03-31,1
2,2023-11-02,6,665917,1.0,2021-05-01,2023-06-30,1
3,2023-11-02,6,665948,1.0,2008-10-01,2024-04-24,1
4,2023-11-02,6,665954,1.0,2020-10-01,2024-04-24,1
...,...,...,...,...,...,...,...
29808,2024-01-20,22,665997,0.5,2008-10-01,2023-06-30,2
29809,2024-01-20,22,665997,0.5,2023-07-01,2024-04-24,2
29810,2024-01-20,22,919208,0.5,2023-12-31,2024-04-24,2
29811,2024-01-20,22,919208,0.5,2016-03-21,2020-03-15,2


In [7]:
hours_associations = hours_associations[(hours_associations['day'] >= hours_associations['from_date']) & (hours_associations['day'] < hours_associations['to_date'])]

In [8]:
hours_associations = pd.merge(hours, hours_associations[['day', 'person_id', 'hour','from_date', 'to_date','store_id']], how='left', on=['day', 'person_id', 'hour'])




In [9]:
work_cal = hours_associations.copy()

In [10]:
work_cal

,day,hour,person_id,worked_hours,from_date,to_date,store_id
0,2023-11-02,6,665917,1.0,2023-07-01,2024-03-31,1.0
1,2023-11-02,6,665948,1.0,2008-10-01,2024-04-24,1.0
2,2023-11-02,6,665954,1.0,2020-10-01,2024-04-24,1.0
3,2023-11-02,6,665986,1.0,2008-10-01,2024-04-24,1.0
4,2023-11-02,6,999012,1.0,2022-10-12,2024-04-24,1.0
...,...,...,...,...,...,...,...
14943,2024-01-20,22,665923,0.5,2008-10-01,2024-04-24,2.0
14944,2024-01-20,22,665926,0.5,2008-10-01,2024-04-24,2.0
14945,2024-01-20,22,665979,0.5,2023-07-01,2024-04-24,2.0
14946,2024-01-20,22,665997,0.5,2023-07-01,2024-04-24,2.0


In [11]:
# work_cal['planned_hours'] = work_cal.groupby(['day','person_id'])['worked_hours_x'].transform('sum')

In [12]:
work_cal.sort_values(by=['day','person_id','hour'],inplace=True)

In [13]:
work_cal.rename(columns={'from_date':'asoc_from_date','to_date':'asoc_to_date'},inplace=True)

In [14]:
incidences['from_date'] = pd.to_datetime(incidences['from_date'])
incidences['to_date'] = incidences['to_date'].fillna(datetime.datetime.today().date())
incidences['to_date'] = pd.to_datetime(incidences['to_date'])
incidences

,person_id,type_name,from_date,to_date
0,665917,Vacaciones,2023-11-20,2023-11-21
1,665917,Vacaciones,2023-11-13,2023-11-19
2,665917,Licencia sin sueldo,2023-12-30,2023-12-31
3,665922,Fin de Semana Libre,2024-01-20,2024-01-21
4,665922,Vacaciones,2023-12-28,2023-12-31
...,...,...,...,...
108,1595955,Festivo cerrado,2023-12-06,2023-12-06
109,1595956,Festivo cerrado,2023-12-06,2023-12-06
110,1595958,Festivo cerrado,2023-12-06,2023-12-06
111,1727011,Vacaciones,2024-01-15,2024-01-21


In [15]:
# work_cal = work_cal[['day','person_id','planned_hours','store_id']].sort_values(by=['person_id','day']).drop_duplicates(subset=['day','person_id'])

In [16]:
work_cal['store_id'] = work_cal['store_id'].fillna('')

In [17]:
work_cal['in_alternate_workplace'] = work_cal.apply(lambda x: True if x['store_id'] != '' else False, axis=1)

In [18]:
work_cal_incidences = pd.merge(work_cal, incidences, how='left', on='person_id')



In [19]:
work_cal_incidences.rename(columns={'from_date':'inc_from_date','to_date':'inc_to_date'},inplace=True)
work_cal_incidences

,day,hour,person_id,worked_hours,asoc_from_date,asoc_to_date,store_id,in_alternate_workplace,type_name,inc_from_date,inc_to_date
0,2023-11-01,15,665923,1.0,2008-10-01,2024-04-24,2.0,True,NaN,NaT,NaT
1,2023-11-01,16,665923,1.0,2008-10-01,2024-04-24,2.0,True,NaN,NaT,NaT
2,2023-11-01,17,665923,1.0,2008-10-01,2024-04-24,2.0,True,NaN,NaT,NaT
3,2023-11-01,18,665923,1.0,2008-10-01,2024-04-24,2.0,True,NaN,NaT,NaT
4,2023-11-01,19,665923,1.0,2008-10-01,2024-04-24,2.0,True,NaN,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...
31897,2024-01-31,20,1305178,1.0,2022-11-09,2024-04-24,1.0,True,Asistencia médica - Día completo,2024-01-31,2024-01-31
31898,2024-01-31,17,1770558,1.0,2024-01-02,2024-04-24,1.0,True,NaN,NaT,NaT
31899,2024-01-31,18,1770558,1.0,2024-01-02,2024-04-24,1.0,True,NaN,NaT,NaT
31900,2024-01-31,19,1770558,1.0,2024-01-02,2024-04-24,1.0,True,NaN,NaT,NaT


In [20]:
work_cal_incidences

,day,hour,person_id,worked_hours,asoc_from_date,asoc_to_date,store_id,in_alternate_workplace,type_name,inc_from_date,inc_to_date
0,2023-11-01,15,665923,1.0,2008-10-01,2024-04-24,2.0,True,NaN,NaT,NaT
1,2023-11-01,16,665923,1.0,2008-10-01,2024-04-24,2.0,True,NaN,NaT,NaT
2,2023-11-01,17,665923,1.0,2008-10-01,2024-04-24,2.0,True,NaN,NaT,NaT
3,2023-11-01,18,665923,1.0,2008-10-01,2024-04-24,2.0,True,NaN,NaT,NaT
4,2023-11-01,19,665923,1.0,2008-10-01,2024-04-24,2.0,True,NaN,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...
31897,2024-01-31,20,1305178,1.0,2022-11-09,2024-04-24,1.0,True,Asistencia médica - Día completo,2024-01-31,2024-01-31
31898,2024-01-31,17,1770558,1.0,2024-01-02,2024-04-24,1.0,True,NaN,NaT,NaT
31899,2024-01-31,18,1770558,1.0,2024-01-02,2024-04-24,1.0,True,NaN,NaT,NaT
31900,2024-01-31,19,1770558,1.0,2024-01-02,2024-04-24,1.0,True,NaN,NaT,NaT


In [21]:
print(datetime.datetime.today().date())

2024-04-24


In [22]:
work_cal_incidences['inc_to_date'].fillna(datetime.datetime.today().date())
work_cal_incidences = work_cal_incidences[(work_cal_incidences['day'] >= work_cal_incidences['inc_from_date']) & (work_cal_incidences['day'] < work_cal_incidences['inc_to_date'])]


In [23]:
work_cal_incidences = pd.merge(work_cal, work_cal_incidences[['day','hour','person_id','type_name','inc_from_date','inc_to_date']], how='left', on=['day','hour','person_id'])

In [24]:
work_cal_incidences

,day,hour,person_id,worked_hours,asoc_from_date,asoc_to_date,store_id,in_alternate_workplace,type_name,inc_from_date,inc_to_date
0,2023-11-01,15,665923,1.0,2008-10-01,2024-04-24,2.0,True,NaN,NaT,NaT
1,2023-11-01,16,665923,1.0,2008-10-01,2024-04-24,2.0,True,NaN,NaT,NaT
2,2023-11-01,17,665923,1.0,2008-10-01,2024-04-24,2.0,True,NaN,NaT,NaT
3,2023-11-01,18,665923,1.0,2008-10-01,2024-04-24,2.0,True,NaN,NaT,NaT
4,2023-11-01,19,665923,1.0,2008-10-01,2024-04-24,2.0,True,NaN,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...
14959,2024-01-31,20,1305178,1.0,2022-11-09,2024-04-24,1.0,True,NaN,NaT,NaT
14960,2024-01-31,17,1770558,1.0,2024-01-02,2024-04-24,1.0,True,NaN,NaT,NaT
14961,2024-01-31,18,1770558,1.0,2024-01-02,2024-04-24,1.0,True,NaN,NaT,NaT
14962,2024-01-31,19,1770558,1.0,2024-01-02,2024-04-24,1.0,True,NaN,NaT,NaT


In [25]:
work_cal_incidences.sort_values(by=['day','person_id','hour'],inplace=True)

In [26]:
'''
 At this stage we have the planned allocation per day and workplace.
 We need to cross reference against worker absences to verify that
 they did indeed work.
'''


'\n At this stage we have the planned allocation per day and workplace.\n We need to cross reference against worker absences to verify that\n they did indeed work.\n'

In [27]:
work_cal_incidences['type_name'] = work_cal_incidences['type_name'].fillna('')

In [28]:
work_cal_incidences['was_absent'] = work_cal_incidences.apply(lambda x: False if x['type_name'] == '' else True, axis=1)

In [29]:
work_cal_incidences['planned_hours'] = work_cal_incidences['worked_hours']

In [30]:
work_cal_incidences['worked_hours'] = work_cal_incidences.apply(lambda x: x['planned_hours'] if x['was_absent'] == False else 0, axis=1)

In [31]:
work_cal_incidences.rename(columns={'type_name':'absence_type'},inplace=True)

In [32]:
work_cal_incidences.to_csv('work_cal_incidences.csv',index=False)

In [33]:
work_cal_incidences

,day,hour,person_id,worked_hours,asoc_from_date,asoc_to_date,store_id,in_alternate_workplace,absence_type,inc_from_date,inc_to_date,was_absent,planned_hours
0,2023-11-01,15,665923,1.0,2008-10-01,2024-04-24,2.0,True,,NaT,NaT,False,1.0
1,2023-11-01,16,665923,1.0,2008-10-01,2024-04-24,2.0,True,,NaT,NaT,False,1.0
2,2023-11-01,17,665923,1.0,2008-10-01,2024-04-24,2.0,True,,NaT,NaT,False,1.0
3,2023-11-01,18,665923,1.0,2008-10-01,2024-04-24,2.0,True,,NaT,NaT,False,1.0
4,2023-11-01,19,665923,1.0,2008-10-01,2024-04-24,2.0,True,,NaT,NaT,False,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14959,2024-01-31,20,1305178,1.0,2022-11-09,2024-04-24,1.0,True,,NaT,NaT,False,1.0
14960,2024-01-31,17,1770558,1.0,2024-01-02,2024-04-24,1.0,True,,NaT,NaT,False,1.0
14961,2024-01-31,18,1770558,1.0,2024-01-02,2024-04-24,1.0,True,,NaT,NaT,False,1.0
14962,2024-01-31,19,1770558,1.0,2024-01-02,2024-04-24,1.0,True,,NaT,NaT,False,1.0
